### Line , edge and shape detection are based on Fourier Transformation

In [1]:
import cv2 as cv
import numpy as np
from scipy import ndimage

In [2]:
kernel_3x3 = np.array([
    [-1,-1,-1],
    [-1,8,-1],
    [-1,-1,-1]
])

In [3]:
kernel_5x5 = np.array([
    [-1,-1,-1,-1,-1],
    [-1,1,2,1,-1],
    [-1,2,4,2,-1],
    [-1,1,2,1,-1],
    [-1,-1,-1,-1,-1]])

In [7]:
img = cv.imread('statue_liberty.jpeg',0)

###  resize the image 
- img = cv.imread('sas.jpg',0)
- new_size = (250, 250) # new_size=(width, height)
- print(f"New height and width: {new_size[1]}, {new_size[0]}" )
- resize_img = cv.resize(img, new_size)

In [8]:
k3 = ndimage.convolve(img,kernel_3x3)
k5 = ndimage.convolve(img,kernel_5x5)

In [ ]:
blurred = cv.GaussianBlur(img,(17,17),0)
g_hpf = img - blurred

In [ ]:
cv.imshow('3x3',k3)
cv.imshow('5x5',k5)
cv.imshow('blurred',blurred)
cv.imshow('g_hpf',g_hpf)
cv.waitKey()
cv.destroyAllWindows()


## CUSTOM Kernels - 

In [ ]:
class VConVolutionFilter(object):
    """ a convolution used for all BGR """
    def __init__(self,kernel):
        self._kernel = kernel
    def apply(self,src,dst):
        """ apply filter a BGR/Gray to source / Destinaton """
        cv.filter2D(src,-1,self._kernel,dst)

In [ ]:
class SharpenFilter(VConVolutionFilter):
    def __init__(self):
        kernel = np.array([
                [-1,-1,-1],
                [-1,9,-1],
                [-1,-1,-1]
                ])
        VConVolutionFilter.__init__(self,kernel)

In [ ]:
class FindEdgesFilter(VConVolutionFilter):
    def __init__(self):
        kernel = np.array([
                [-1,-1,-1],
                [-1,8,-1],
                [-1,-1,-1]
                ])
        VConVolutionFilter.__init__(self,kernel)

In [ ]:
class BlurFilter(VConVolutionFilter):
    def __init__(self):
        kernel = np.array([
            [0.04,0.04,.04,.04,.04],
              [0.04,0.04,.04,.04,.04],
              [0.04,0.04,.04,.04,.04],
              [0.04,0.04,.04,.04,.04],
              [0.04,0.04,.04,.04,.04]])
        VConVolutionFilter.__init__(self,kernel)
        
            
            

In [ ]:
class EmbossFilter(VConVolutionFilter):
    def __init__(self):
        kernel = np.array([
            [-2,-1,0],
            [-1,1,1],
            [0,1,2]])
        VConVolutionFilter.__init__(self,kernel)

## Edge detection with Canny

In [ ]:
import cv2 as cv 
import numpy as np

img = cv.imread('statue_liberty.jpeg',0)
cv.imwrite('canny.jpg',cv.Canny(img,200,300))
cv.imshow('statue liber',img)
cv.imshow('canny statue',cv.imread('canny.jpg'))
cv.waitKey()
cv.destroyAllWindows()

# Countour Detection
- a black image create 200x200 and then the edge detection by red colour 
- cv.RETR_TREE retrive the entire hierarchy of external and internal contours

In [56]:
import cv2 as cv 
import numpy as np


In [57]:
img = np.zeros((200,200),dtype=np.uint8)
img[50:150,50:150]=255

In [58]:
ret,thresh = cv.threshold(img,127,255,0)
contours,hirerar = cv.findContours(thresh,cv.RETR_TREE,cv.CHAIN_APPROX_SIMPLE)

In [59]:
color = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
img = cv.drawContours(color,contours,-1,(0,0,255),2)

In [54]:
cv.imshow('contours',color)
cv.waitKey()
cv.destroyAllWindows()

# convex contours and the Douglas Peucker Algorithm
- Contours can be explained simply as a curve joining all the continuous points (along the boundary), having same color or intensity.

In [15]:
import cv2 as cv
import numpy as np


In [16]:
# test - 01 
# img = cv.pyrDown(cv.imread("enclose.PNG",cv.IMREAD_UNCHANGED))
# test - 02
img = cv.pyrDown(cv.imread("statue_liberty.jpeg",cv.IMREAD_UNCHANGED))


In [17]:
# img
ret , thresh = cv.threshold(cv.cvtColor(img,cv.COLOR_BGR2GRAY),127,255,cv.THRESH_BINARY)

In [18]:
contours,hier =  cv.findContours(thresh,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)

In [19]:
black = np.zeros_like(img)

In [20]:
 for cnt in contours:
        epsilon = 0.01 * cv.arcLength(cnt,True)
        approx = cv.approxPolyDP(cnt,epsilon,True)
        hull = cv.convexHull(cnt)
        
        cv.drawContours(black,[cnt],-1,(0,255,0),2)
        cv.drawContours(black,[approx],-1,(255,255,0),2)
        cv.drawContours(black,[hull],-1,(0,0,255),2)
        

cv.imshow("hull",black)
cv.waitKey()
cv.destroyAllWindows()

# Detecting Lines 

In [ ]:
img = cv.imread('line.png')
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
edges = cv.Canny(gray,50,120)
minLength = 20
maxLineGap = 5

lines = cv.HoughLinesP(edges,1,np.pi/180.0,20,minLength,maxLineGap)

for x1,y1,x2,y2 in lines[0]:
    cv.line(img,(x1,y1),(x2,y2),(0,255,0),2)

cv.imshow('edges',edges)
cv.imshow('lines',img)
cv.waitKey()
cv.destroyAllWindows()

# Detecing Circles 

In [36]:
import cv2 as cv
import numpy as np


In [37]:
planet = cv.imread('planet.PNG')
gray_image = cv.cvtColor(planet,cv.COLOR_BGR2GRAY)
gray_image = cv.medianBlur(gray_image,5)

In [38]:
circles = cv.HoughCircles(gray_image,cv.HOUGH_GRADIENT,1,120,param1=100,param2=30,minRadius=0,maxRadius=0)

In [39]:
circles = np.uint16(np.around(circles))

In [43]:
for i in circles[0,:]:
    # draw the output circle 
    cv.circle(planet,(i[0],i[1]),i[2],(0,255,0),2)
    # draw the center of the circle 
    cv.circle(planet,(i[0],i[1]),2,(0,0,255),3)

cv.imwrite('planets_circle.jpg',planet)
cv.imshow('hough circle',planet)
cv.waitKey()
cv.destroyAllWindows()